In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import random
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service as ChromeService

##### Ejercicio 1
Realiza web scraping de dos de las tres páginas web propuestas utilizando BeautifulSoup primero y Selenium después. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (haz alguna búsqueda primero y chasquea algún contenido)

##### El scraping. 
En el contexto de la informática, se refiere al proceso de extracción automatizada de datos de páginas web. Se utiliza por diversas razones en diferentes campos, entre las cuales se incluyen:

Recopilación de información: El scraping se utiliza para extraer datos relevantes de páginas web para su posterior análisis. Esto puede incluir la extracción de noticias, precios de productos, información de eventos, datos de redes sociales, entre otros.

Monitoreo y seguimiento: Las empresas utilizan el scraping para monitorear la presencia en línea de sus productos o servicios, así como para recopilar información sobre la competencia. Por ejemplo, pueden rastrear los precios de los competidores, las reseñas de los clientes, etc.

Investigación y análisis: En campos como la investigación académica, el periodismo de datos y el análisis de mercado, el scraping se utiliza para recopilar grandes cantidades de datos de diferentes fuentes en línea para su posterior análisis y estudio.

Automatización de tareas: En algunos casos, el scraping se utiliza para automatizar tareas repetitivas en línea, como la recopilación de información de múltiples fuentes o la actualización de bases de datos.

Creación de servicios y productos: Algunas empresas utilizan el scraping como parte de sus servicios o productos, ofreciendo herramientas que permiten a los usuarios extraer datos de la web de manera automatizada para diversos fines.

Es importante tener en cuenta que el scraping debe realizarse de manera ética y respetando los términos de servicio de los sitios web, evitando causar daños o sobrecargar los servidores de los sitios objetivo.

#### 'http://quotes.toscrape.com/'

##### Beautiful Soup 
es una biblioteca de Python que se utiliza comúnmente para extraer datos de archivos HTML y XML. Permite analizar y manipular el contenido de una página web de manera sencilla y eficiente.

Con BeautifulSoup, puedes buscar, filtrar y manipular elementos HTML y XML utilizando métodos y atributos similares a los que encontrarías en un navegador web. Esto facilita la extracción de datos específicos de una página web, como texto, enlaces, imágenes, tablas y mucho más.

Beautiful Soup también es muy flexible y puede manejar HTML y XML mal formados, lo que lo convierte en una herramienta robusta para el scraping web en una variedad de situaciones.

In [6]:
url = 'http://quotes.toscrape.com/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

Declaro url con el link de la pagina a la que quiero hacer web scraping, luego solicito permiso para acceder a datos de la url espcecificada.

In [7]:
quotes = soup.find_all('span', class_='text')
authors = soup.find_all('small', class_='author')


if response.status_code == 200:
    for quote, author in zip(quotes, authors):
        print(f"{author.encode_contents()}: {quote.encode_contents()}")
else:
    print(f'Couldn\'t retrieve the page. Status code: {response.status_code}')

b'Albert Einstein': b'\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.\xe2\x80\x9d'
b'J.K. Rowling': b'\xe2\x80\x9cIt is our choices, Harry, that show what we truly are, far more than our abilities.\xe2\x80\x9d'
b'Albert Einstein': b'\xe2\x80\x9cThere are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.\xe2\x80\x9d'
b'Jane Austen': b'\xe2\x80\x9cThe person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.\xe2\x80\x9d'
b'Marilyn Monroe': b"\xe2\x80\x9cImperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.\xe2\x80\x9d"
b'Albert Einstein': b'\xe2\x80\x9cTry not to become a man of success. Rather become a man of value.\xe2\x80\x9d'
b'Andr\xc3\xa9 Gide': b'\xe2\x80\x9cIt is better to be hated for what you are than to be loved for what you are not.\xe

- Declaramos soup para representar la estructura del contenido del HTML.
- Declaramos quotes, para ingresar a los elementos Span que contiene la clase text y asi obtener el texto.
- Declaramos authors, para ingresar a los elementos small que ocntienen la clase authors, y asi obtener el autor de cada texto. 

Se debe analizar la pagina web, las etiquetas, elementos y las clase para acceder a ellas correctamente y extrae la informacion correcta. 

In [8]:
dfsoup = pd.DataFrame({'Author': [author.text for author in authors],
                   'Quote': [quote.text for quote in quotes]})
dfsoup

,Author,Quote
0,Albert Einstein,“The world as we have created it is a process ...
1,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
2,Albert Einstein,“There are only two ways to live your life. On...
3,Jane Austen,"“The person, be it gentleman or lady, who has ..."
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
5,Albert Einstein,“Try not to become a man of success. Rather be...
6,André Gide,“It is better to be hated for what you are tha...
7,Thomas A. Edison,"“I have not failed. I've just found 10,000 way..."
8,Eleanor Roosevelt,“A woman is like a tea bag; you never know how...
9,Steve Martin,"“A day without sunshine is like, you know, nig..."


Los datos que se extrajeron fueron solo para la primera pagina y su contenido. por tanto vamos a hacer un cliclo for para recorrer las 10 paginas. 

In [9]:
base_url = 'http://quotes.toscrape.com/page/'

all_quotes = []

for page_number in range(1, 11):  # 10 páginas en total
    # Construir la URL de la página actual
    url = base_url + str(page_number)
    
    # Realizar la solicitud HTTP GET
    response = requests.get(url)
    
    # Verificacion de solicitud
    if response.status_code == 200:
        # Parsear el contenido HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrar todas las citas y autores en la página actual
        quotes = [quote.text for quote in soup.find_all('span', class_='text')]
        authors = [author.text for author in soup.find_all('small', class_='author')]
        
        # Almacenar las citas y los autores en la lista
        all_quotes.extend(zip(authors, quotes))
    else:
        print(f'Couldn\'t retrieve page {page_number}. Status code: {response.status_code}')

# Crear DataFrame de pandas
dfsoup2= pd.DataFrame(all_quotes, columns=['Author', 'Quote'])

dfsoup2

,Author,Quote
0,Albert Einstein,“The world as we have created it is a process ...
1,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
2,Albert Einstein,“There are only two ways to live your life. On...
3,Jane Austen,"“The person, be it gentleman or lady, who has ..."
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
...,...,...
95,Harper Lee,“You never really understand a person until yo...
96,Madeleine L'Engle,“You have to write the book that wants to be w...
97,Mark Twain,“Never tell the truth to people who are not wo...
98,Dr. Seuss,"“A person's a person, no matter how small.”"


##### Selenium

Selenium proporciona una API que te permite controlar los navegadores web más populares, como Chrome, Firefox, Safari, e Internet Explorer, entre otros. Puedes enviar comandos al navegador, simular acciones del usuario y recuperar información de la página web.

En resumen, Selenium es una herramienta poderosa y versátil que se utiliza ampliamente en el desarrollo de software y la automatización de pruebas para interactuar con aplicaciones web de manera programática.

Selenium puede usarse para realizar scraping web, pero no es la herramienta más eficiente para este propósito. Selenium está diseñado principalmente para la automatización de pruebas y la interacción con aplicaciones web como un usuario humano haría en un navegador real.

Si bien es posible utilizar Selenium para extraer datos de una página web (es decir, realizar scraping web), existen otras herramientas específicamente diseñadas para esta tarea que son más eficientes y fáciles de usar, como BeautifulSoup y Scrapy en Python. Estas herramientas están optimizadas para analizar y extraer datos de páginas web de una manera más eficiente y menos costosa computacionalmente que Selenium.

In [56]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)
url = 'http://quotes.toscrape.com/'
browser.get(url)

In [57]:
quotes=browser.find_elements(By.CLASS_NAME, 'quote')

In [58]:
for quote in quotes:
    text=quote.find_element(By.CLASS_NAME, 'text').text
    author=quote.find_element(By.CLASS_NAME, 'author').text
    print(author, text)

Albert Einstein “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
J.K. Rowling “It is our choices, Harry, that show what we truly are, far more than our abilities.”
Albert Einstein “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Jane Austen “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Marilyn Monroe “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Albert Einstein “Try not to become a man of success. Rather become a man of value.”
André Gide “It is better to be hated for what you are than to be loved for what you are not.”
Thomas A. Edison “I have not failed. I've just found 10,000 ways that won't work.”
Eleanor Roosevelt “A woman is like a tea bag; you never know how strong it is until it's in hot water.”
Steve Martin

In [59]:
dfsel = pd.DataFrame(columns=['author', 'text'])

In [60]:
for i in range(1,11):
    url="https://quotes.toscrape.com/page/" + str(i) + "/"
    browser.get(url)
    quotes=browser.find_elements(By.CLASS_NAME, 'quote')
    for quote in quotes:
        text=quote.find_element(By.CLASS_NAME, 'text').text
        author=quote.find_element(By.CLASS_NAME, 'author').text
        dfsel2 = pd.concat([dfsel2, pd.DataFrame([{'author':author, 'text': text}])], ignore_index=True)
browser.quit()

In [61]:
dfsel2.head(10)

,author,text
0,George R.R. Martin,“... a mind needs books as a sword needs a whe...
1,Albert Einstein,“The world as we have created it is a process ...
2,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
3,Albert Einstein,“There are only two ways to live your life. On...
4,Jane Austen,"“The person, be it gentleman or lady, who has ..."
5,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
6,Albert Einstein,“Try not to become a man of success. Rather be...
7,André Gide,“It is better to be hated for what you are tha...
8,Thomas A. Edison,"“I have not failed. I've just found 10,000 way..."
9,Eleanor Roosevelt,“A woman is like a tea bag; you never know how...


#### https://es.wikipedia.org/wiki/Marina_Abramovi%C4%87

##### Beautiful Soup

Para la pagina Wikipedia con contenido de Marina Abramovic, quiero escrapear la infobox, que contiene los datos relevantes. y convertirlos en una dataframe.

In [62]:
url="https://es.wikipedia.org/wiki/Marina_Abramovi%C4%87"
response = requests.get(url)

In [63]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    infobox = soup.find('table', class_='infobox')
else:
    print("Error al realizar la solicitud GET:", response.status_code)

In [64]:
infobox

<table class="infobox biography vcard" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;"><tbody><tr><th class="cabecera persona" colspan="3" style="text-align:center;background-color:transparent;color:inherit;">Marina Abramović</th></tr><tr><td class="imagen" colspan="3" style="text-align:center;line-height:1.3em; vertical-align:middle;;">
<span typeof="mw:File"><a class="mw-file-description" href="/wiki/Archivo:Marina_Abramovi%C4%87._The_Cleaner_(45524492341).jpg"><img class="mw-file-element" data-file-height="1727" data-file-width="1386" decoding="async" height="274" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/34/Marina_Abramovi%C4%87._The_Cleaner_%2845524492341%29.jpg/220px-Marina_Abramovi%C4%87._The_Cleaner_%2845524492341%29.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/34/Marina_Abramovi%C4%87._The_Cleaner_%2845524492341%29.jpg/330px-Marina_Abramovi%C4%87._The_Cleaner_%2845524492341%29.jpg 1.5x, //upload.wikimedia.org/wikipedia/com

Con un condiconal traemos la información que queremos (infobox), sin embargo al traerla, tambien se veran las etiquetas, parentesis, corchetes, informacion no organizada, por tanto, eliminamos elementos del texto que no necesitamos y organizamos la informacion en un dataframe. 

In [65]:
if infobox: # Si se encuentra la infobox, extraer los datos y convertirlos en un DataFrame
        infobox_data = {} # Crear un diccionario para almacenar los datos de la infobox
        
        for row in infobox.find_all('tr'): # Iterar sobre las filas de la infobox
            # Obtener el texto de la etiqueta de encabezado (th) y el texto de la etiqueta de datos (td)
            header = row.find('th')
            data = row.find('td')
            
            # Si se encuentran tanto el encabezado como los datos, agregarlos al diccionario
            if header and data:
                # Eliminar los posibles corchetes de referencia
                header_text = header.get_text(strip=True).replace('[', '').replace(']', '')
                data_text = data.get_text(strip=True).replace('[', '').replace(']', '')
                
                # Agregar los datos al diccionario
                infobox_data[header_text] = data_text
        
        # Convertir el diccionario en un DataFrame de pandas
        dfsoupMarina = pd.DataFrame(list(infobox_data.items()), columns=['Datos', 'Descripcion'])
        
        # Imprimir el DataFrame
        print(dfsoupMarina)
else:
        print("No se encontró la infobox en la página.")

           Datos                                        Descripcion
0     Nacimiento  30 de noviembre de 1946(77 años)Belgrado(Repúb...
1     Residencia                                         Nueva York
2   Nacionalidad                                             Serbia
3        Cónyuge  UlayNeša Paripović(1971-1976)Paolo Canevari(20...
4     Educada en  Academy of Fine Arts, Belgrade(hasta 1970)Acad...
5      Ocupación  Artista de performance,fotógrafa, videoartista...
6           Área  Performance, performance artwork,arte corporal...
7   Conocida por                                        Performance
8      Empleador  Academia de Bellas ArtesUniversidad de las Art...
9     Movimiento                       Arte conceptualy performance
10     Seudónimo                                  Abramović, Marina
11        Género                                        Performance
12    Miembro de                              Real Academia de Arte
13     Sitio web                            www.

Con este condicional, creamos un diccionario para guardar la busqueda de encabezados 'th', con datos 'td', luego  limpiamos el texto de signos que no queremos obtener y organizamos la información en un dataframe. 

##### Selenium

##### Primer intento

In [ ]:
url="https://es.wikipedia.org/wiki/Marina_Abramovi%C4%87"
options = Options()
options.add_argument('-headless=new')
browser = webdriver.Chrome(options=options)
browser.get(url)

In [ ]:
infobox2=browser.find_elements(By.CLASS_NAME, 'infobox')

Busco los elementos con el metodo find_elements (By.CLASS_NAME, "infobox") que lo obtenego de inspeccionar la pagina de wikipedia.

In [ ]:
if infobox2:
    count=0
    for infobox in infobox2:
        print(count,infobox.text)
        count+=1
else:
    print("No se encontraron infoboxes en la página.")

Hago un ciclo for para saber si encontro algun elemento infobox en la pagina de wikipedia.

In [ ]:
headers=browser.find_elements(By.TAG_NAME, "th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

In [ ]:
df=pd.DataFrame(columns=columns)

Tengo toda la informacion del infobox de wikipedia, pero quiero hacer un dataframe de esta informacion, sin embargo intento convertirlo en dataframe, pero que pasa que el infobox tienen encabezados, subtitulos. Estos titulos y subtitulos estan clasificados como 'th', pero cuando se hacia el dataframe me resulta, que no hay correspondencia de datos para este encabezado, por que? por que no es uncabezado que contiene información, sino un simple subtitulo, por tanto era una celda que apareceria como Nan. por tanto es complejo traer este tipo de informacion con selenium. puede haber formas de hacerlo si, pero no lo logre encontrar.

Nota: pongo esta información del proceso que hice, por que siento que es valida para saber a la hora de scrapear, que herramientas son buenas en que casos. y por mi experiencia veo que selenium no es muy buena, si el HTML, no tiene una esctrutura definida, como una tabla. 

##### Segundo intento

Ya que Selenium, me ha hecho pensar que si el HTML, no tiene una estructura definida, sera dificil configurar la informacion en un dataframe, pues escojo una pagina web que contenga tablas sencillas que si pueda consultar y extraer.

In [15]:
url="https://es.wikipedia.org/wiki/Anexo:Pel%C3%ADculas_con_las_mayores_recaudaciones"
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)
browser.get(url)

In [16]:
tables = browser.find_elements(By.XPATH,"//table[@class='wikitable sortable jquery-tablesorter']")

para selenium con el metodo find_elements se encuentran multiples elementos, tuve inconveniente con este metodo por que tambien se puede usar find_element, que es para encontrar el primer elemento, esta pagina tenia multiples tablas y yo queria la segunda, por tanto el adecuado es find_elements, por otro lado intente encontrar por class_name la tabla, pero al poner la clase 'wikitable sortable jquery-tablesorter', que es la clase correcta inspeccionando wikipedia, no encontraba las tablas, por tanto tuve que hacer una busqueda mucho mas robusta con Xpath, que si las encontró.

In [17]:
mas_taquilleras=tables[1]

ahora que con Xpath, me ha encontrado la clase de tablas, debo escoger la tabla que quiero, la segunda por tanto [1]. y almacenarla.

In [18]:
table_html = mas_taquilleras.get_attribute('outerHTML')
browser.quit()
df_pelis= pd.read_html(table_html)[0]
df_pelis

,N.º,Título,Distribuidora(s),Recaudación mundial en USD,Año,Director(es)
0,1,Lo que el viento se llevó,MGM,4 192 000 000,1939,Victor Fleming
1,2,Avatar,20th Century Fox,3 824 000 000,2009,James Cameron
2,3,Titanic,20th Century Fox,3 485 000 000,1997,James Cameron
3,4,Star Wars: Episodio IV - Una nueva esperanza,20th Century Fox,3 443 000 000,1977,George Lucas
4,5,Avengers: Endgame,Disney,3 165 000 000,2019,Anthony y Joe Russo
5,6,The Sound of Music,20th Century Fox,2 884 000 000,1965,Robert Wise
6,7,"E.T., el extraterrestre",Universal Pictures,2 815 000 000,1982,Steven Spielberg
7,8,Los diez mandamientos,Paramount Pictures,2 665 000 000,1956,Cecil B. DeMille
8,9,Doctor Zhivago,MGM,2 526 000 000,1967,David Lean
9,10,Star Wars: Episodio VII - El despertar de la F...,Disney,2 491 000 000,2015,J. J. Abrams


Para obtener el elemento  tabla con selenium Webdrive, uso el metodo get_attribute('outerHTML'), luego cierro el navegador y convierto la tabla en un Dataframe.

Pero quiero saber si solo puedo consultar con el metodo, By.XPATH, o puedo usar otros, pues la verdad Selenioum no me ha parecido lo mas facil. asi que voy a hacer un intento con otro metodo diferente, (BY.TAG_NAME)

##### Tercer intento

In [19]:
driver = webdriver.Chrome()
url = "https://es.wikipedia.org/wiki/Anexo:Pel%C3%ADculas_con_las_mayores_recaudaciones"
driver.get(url)

primera_tabla = driver.find_elements(By.TAG_NAME, 'table')[0]

if primera_tabla:
    table_html = primera_tabla.get_attribute('outerHTML')
    
    df_may_reca = pd.read_html(table_html, encoding='utf-8')[0]
    print(df_may_reca)
else:
    print("No se encontró la primera tabla en la página.")
driver.quit()

    N.º                                           Película  \
0     1                                             Avatar   
1     2                                  Avengers: Endgame   
2     3                           Avatar: The Way of Water   
3     4                                            Titanic   
4     5  Star Wars: Episodio VII - El despertar de la F...   
..  ...                                                ...   
95   96                                            Hi, Mom   
96   97                                   Independence Day   
97   98                                               Coco   
98   99          Animales fantásticos y dónde encontrarlos   
99  100                                      Shrek Tercero   

    Recaudación mundial Taquilla (EE. UU.)  Taquilla (fuera de EE. UU.)  \
0                   NaN                NaN                          NaN   
1                   NaN                NaN                          NaN   
2                   NaN       

Fue posible usando (BY.TAG_NAME) se hizo la consulta y se pudo traer la tabla

In [20]:
df_may_reca

,N.º,Película,Recaudación mundial,Taquilla (EE. UU.),Taquilla (fuera de EE. UU.),Presupuesto,Distribuidora(s),Año de estreno,Director(es)
0,1,Avatar,NaN,NaN,NaN,NaN,20th Century Fox,2009,James Cameron
1,2,Avengers: Endgame,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2019,Anthony y Joe Russo
2,3,Avatar: The Way of Water,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2022,James Cameron
3,4,Titanic,NaN,NaN,NaN,NaN,20th Century Fox / Paramount Pictures,1997,James Cameron
4,5,Star Wars: Episodio VII - El despertar de la F...,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2015,J. J. Abrams
...,...,...,...,...,...,...,...,...,...
95,96,"Hi, Mom",NaN,—,NaN,NaN,Beijing Jingxi Culture & Tourism Co,2021,Ling Jia
96,97,Independence Day,NaN,NaN,NaN,NaN,20th Century Fox,1996,Roland Emmerich
97,98,Coco,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2017,Lee Unkrich / Adrián Molina
98,99,Animales fantásticos y dónde encontrarlos,NaN,NaN,NaN,NaN,Warner Bros.,2016,David Yates


Pero, para algunas columnas no me ha traido la infromacion por tanto me muestra Nans, esto puede deberse a la estructura original de la tabla en wikipedia. pues estos datos que no me extraen tienen un style de display:none. lo que significa que la naturaleza de este contenido no la quieren accesible o manipulable por parte de wikipedia. 

##### Cuarto intento

In [159]:
url="https://es.wikipedia.org/wiki/Anexo:Pel%C3%ADculas_con_las_mayores_recaudaciones"
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)
browser.get(url)

In [22]:
tables = browser.find_elements(By.XPATH,"//table[@class='wikitable sortable jquery-tablesorter']")

In [23]:
mayores_recaudos=tables[0]

In [24]:
table_html = mayores_recaudos.get_attribute('outerHTML')
browser.quit()
df2_may_recau = pd.read_html(table_html)[0]
df2_may_recau

,N.º,Película,Recaudación mundial,Taquilla (EE. UU.),Taquilla (fuera de EE. UU.),Presupuesto,Distribuidora(s),Año de estreno,Director(es)
0,1,Avatar,NaN,NaN,NaN,NaN,20th Century Fox,2009,James Cameron
1,2,Avengers: Endgame,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2019,Anthony y Joe Russo
2,3,Avatar: The Way of Water,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2022,James Cameron
3,4,Titanic,NaN,NaN,NaN,NaN,20th Century Fox / Paramount Pictures,1997,James Cameron
4,5,Star Wars: Episodio VII - El despertar de la F...,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2015,J. J. Abrams
...,...,...,...,...,...,...,...,...,...
95,96,"Hi, Mom",NaN,—,NaN,NaN,Beijing Jingxi Culture & Tourism Co,2021,Ling Jia
96,97,Independence Day,NaN,NaN,NaN,NaN,20th Century Fox,1996,Roland Emmerich
97,98,Coco,NaN,NaN,NaN,NaN,Walt Disney Studios Motion Pictures,2017,Lee Unkrich / Adrián Molina
98,99,Animales fantásticos y dónde encontrarlos,NaN,NaN,NaN,NaN,Warner Bros.,2016,David Yates


Intento traer la misma tabla para verificar si es posible, con un metodo mas robusco (Xpath) traer los datos completos y no fue posible, igualmente las mismas columnas siguen estando con datos NaN. esto podria deberse a que la estructura original de la tabla es inesperada para Pandas. pero esto es debido a la naturaleza de los datos. 

#### Ejercicio 2

Realiza web scraping de dos de las tres páginas web propuestas utilizando BeautifulSoup primero y Selenium después. 

In [25]:
dfsoup2

,Author,Quote
0,Albert Einstein,“The world as we have created it is a process ...
1,J.K. Rowling,"“It is our choices, Harry, that show what we t..."
2,Albert Einstein,“There are only two ways to live your life. On...
3,Jane Austen,"“The person, be it gentleman or lady, who has ..."
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and..."
...,...,...
95,Harper Lee,“You never really understand a person until yo...
96,Madeleine L'Engle,“You have to write the book that wants to be w...
97,Mark Twain,“Never tell the truth to people who are not wo...
98,Dr. Seuss,"“A person's a person, no matter how small.”"


### Quotes to Scrape
#### About Dataset:  
##### Context: 
Este conjunto de datos contiene citas de diferentes autores. para cada fila es una cita y un autor diferente. 
##### Content: 
100 filas x 2 columnas.(descritas a continuación) 
- Author: Contiene los disferentes autores
- Quote: Contiene las citas de los autores. 
##### references: http://quotes.toscrape.com/

##### Data: 
14-02-2023

In [66]:
dfsoupMarina

,Datos,Descripcion
0,Nacimiento,30 de noviembre de 1946(77 años)Belgrado(Repúb...
1,Residencia,Nueva York
2,Nacionalidad,Serbia
3,Cónyuge,UlayNeša Paripović(1971-1976)Paolo Canevari(20...
4,Educada en,"Academy of Fine Arts, Belgrade(hasta 1970)Acad..."
5,Ocupación,"Artista de performance,fotógrafa, videoartista..."
6,Área,"Performance, performance artwork,arte corporal..."
7,Conocida por,Performance
8,Empleador,Academia de Bellas ArtesUniversidad de las Art...
9,Movimiento,Arte conceptualy performance


### Wikipedia - Marina Abramović
#### About Dataset:  
##### Context: 
Este conjunto de datos contiene el infobox descriptivo de la biografia de la artista Marina Abramović. los datos, mas relevantes de su vida y su carrera como artista. 
##### Content: 
14 filas x 2 columnas.(descritas a continuación) 
- Datos: Contiene el titulo del dato
- Descripcion: Contiene la descripcion del dato o información correspondiente a Dato. 
##### references: https://es.wikipedia.org/wiki/Marina_Abramovi%C4%87

##### Data: 
16-02-2023

In [ ]:
df_pelis

### Películas con las mayores recaudaciones
#### About Dataset:  
##### Context: 
Este conjunto de datos contiene la informacion de las Películas más taquilleras por ajuste de inflación hasta 2022. 

##### Content: 
10 filas x 5 columnas.(descritas a continuación) 
- N.º: El enumerador de las peliculas, segun su recaudo. de forma descendente. 
- Título: Contiene el nombre de la pelicula
- Distribuidora(s)	: Contiene el nombre del estudio que realizo la pelicula y los comercializa.. 
- Recaudación mundial en USD: Contiene el valor que recaudo la pelicula, respecto a la inflacion, lo que se traduce en que las peliculas antiguas donde una entrada costaba diferente a lo que cuesta hoy, o los formatos donde las peliculas tuvbieron mayor recaudo, son diferentes para las diferentes pelciuulas, algunas trinfaron en 3d, IMAX, y otras en formatos estandar. por tanto este  conjunto de datos trata de homogenizar las variables. sin embargo hay factores que aun no son tenido en cuenta. como la inflacion a nuvel mundial. ya que es un dato exporadito y dinamico para cada pais. 
- Año: Año de lanzamiento.
- Director(es): Nombre del director de cada pelicula.

##### references: https://es.wikipedia.org/wiki/Marina_Abramovi%C4%87

##### Data: 
18-02-2023

#### Ejercicio 3
Elige una página web que quieras y realiza web scraping mediante la librería Selenium primero y Scrapy después. 

Decido hacer scraping a la pagina de IBEX 35, para extraer una tabla, esta pagina al ser dinamica, me permite hacer scraping con Selenium.

In [174]:
options = Options()
url = "https://www.bolsasymercados.es/bme-exchange/es/Mercados-y-Cotizaciones/Acciones/Mercado-Continuo/Precios/ibex-35-ES0SI0000005"
browser = webdriver.Chrome(options=options)
browser.get(url)
time.sleep(5)

Configuro Selenium e inicializo en navegador. agrego ademas un time.sleep para permitir que la pagina cargue completamente y asi poder extraer los datos. 

In [175]:
header_row = browser.find_element(By.XPATH, "//table[contains(@class, 'shares-table')]/thead/tr")
header_cells = header_row.find_elements(By.TAG_NAME, "th")
column_names = [cell.text for cell in header_cells]

rows = browser.find_elements(By.XPATH, "//table[contains(@class, 'shares-table')]/tbody/tr")

data = []

for row in rows:
    row_data = []
    cells = row.find_elements(By.TAG_NAME, "td")
    for cell in cells:
        row_data.append(cell.text)
    data.append(row_data)

df = pd.DataFrame(data, columns=column_names)

browser.quit()

Accedo al elemento que quiero (tabla) con By.Xpath tanto para encontrar encabezados como para filas, y luego creo una lista para almacenar los datos de la tabla, luego con un for itero sobre las filas y extraigo los datos de cada celda con (By.Tag_Name) y luego convierto los datos en un dataframe. cierro el navegador. 

In [176]:
df

,Nombre,Último,% Dif.,Máximo,Mínimo,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"108,0000","0,09%","109,1500","107,9000",16.978,"1.840,18",22/02/2024,11:37:06
1,ACCIONA ENER,"20,7600","0,78%","20,8000","20,5400",29.252,"605,94",22/02/2024,11:35:59
2,ACERINOX,"10,2600","-0,24%","10,3550","10,2450",153.011,"1.575,26",22/02/2024,11:38:42
3,ACS,"37,0100","0,76%","37,3400","36,9000",76.699,"2.841,02",22/02/2024,11:35:03
4,AENA,"172,0500","0,38%","173,4000","171,7500",16.862,"2.904,56",22/02/2024,11:39:41
5,AMADEUS,"61,4400","0,23%","62,6400","61,3800",105.606,"6.542,51",22/02/2024,11:38:49
6,ARCELORMIT.,"24,3250","-0,14%","24,6800","24,3000",66.126,"1.621,77",22/02/2024,11:39:32
7,B.SANTANDER,"3,8455","0,76%","3,8695","3,8205",18.306.504,"70.327,87",22/02/2024,11:39:34
8,BA.SABADELL,"1,1910","2,36%","1,1940","1,1675",11.160.968,"13.175,29",22/02/2024,11:39:19
9,BANKINTER,"5,8060","1,36%","5,8060","5,7480",940.625,"5.439,92",22/02/2024,11:39:22


Ahora tenemos un dataset con encabezados de columnas que representan informacion y filas que contienen datos de cada empresa en la bolsa de valores. 

### Precios - Bolsa de Valores Ibex 35
#### About Dataset:  
##### Context: 
Tenemos un dataset con encabezados de columnas que representan el tipo de información y filas que contienen datos de cada empresa en la bolsa de valores.
##### Content: 
35 filas x 9 columnas.(descritas a continuación) 

Nombre: El nombre de la empresa o entidad cuyas acciones se están negociando.
Último: El precio más reciente al que se negoció la acción.
% Dif.: La variación porcentual del precio en comparación con el precio de cierre anterior.
Máximo: El precio máximo al que se negoció la acción durante el período.
Mínimo: El precio mínimo al que se negoció la acción durante el período.
Volumen: La cantidad total de acciones negociadas durante el período.
Efectivo (miles €): El valor total de las acciones negociadas en euros, en miles.
Fecha: La fecha en que se registraron los datos.
Hora: La hora en que se registraron los datos.

##### references: https://www.bolsasymercados.es/bme-exchange/es/Mercados-y-Cotizaciones/Acciones/Mercado-Continuo/Precios/ibex-35-ES0SI0000005

##### Data: 
22-02-2023